# Evaluating All Models Using GPT-4 as an A/B Tester

## Introduction
This notebook contains code to evaluate all A/B tests across all prompts for a benchmark. The code reads in a list of unevaluated AB tests with prompts and responses from a database, runs parallel A/B tests with GPT-4 as a judge, and writes the results back to the database.

## Setup
Before running the code, make sure to install the necessary dependencies by setting up the Poetry environment. You will also need to set up a database connection by filling in the appropriate values in the cells below. You will also need to provide up an OpenAI API key and organization ID (if appropriate).

Additionally, specify the benchmark and evaluating model to use for evaluation. The benchmark is the name of the benchmark to evaluate. The evaluating model is the name of the model to use as a judge for the A/B tests. The evaluating model must be a model that is already in the database.

In [1]:
import os

In [2]:
os.environ['OPENAI_API_KEY']="YOUR_API_KEY"
os.environ['OPENAI_ORGANIZATION']="YOUR_ORG_ID"

In [3]:
os.environ['POSTGRES_HOST']="localhost"
os.environ['POSTGRES_DB']="llm_eval_db"
os.environ['POSTGRES_USER']="maker"
os.environ['POSTGRES_PASSWORD']="makerpassword"

In [4]:
EVALUATING_MODEL = "gpt-4-0613"

## Code Structure
The code is split into several sections:

### Imports
The first section imports the necessary modules and functions.

### Data Loading
The second section loads the unevaluated AB test prompts from the database and converts them to requests for the OpenAI API.

### Model Evaluation
The third section evaluates the AB test prompts using the OpenAI cookbook.

### Data Writing
The fourth section parses the evaluation results and writes them back to the database.

## Usage
To use this code, simply run the notebook from start to finish. The results will be written back to the database automatically.

In [5]:
from db.db_utils import get_all_unevaluated_ab_test_prompts_for_benchmark, insert_evals_by_model_into_db
from src.parse import db_game_to_dict, construct_request, parse_result_to_eval_entry, games_list_to_lookup
from src.utils import read_jsonl, write_jsonl

## Data Loading

In [6]:
incomplete_game_rows = await get_all_unevaluated_ab_test_prompts_for_benchmark(eval_model_name=EVALUATING_MODEL)
print(f"{len(incomplete_game_rows)} incomplete games found")
if len(incomplete_game_rows) > 0:
    print(f"Example: {incomplete_game_rows[0]}")

1 incomplete games found
Example: (UUID('76314a10-f32a-4188-bba8-fd56f0ba95f4'), UUID('10596d4c-4bbe-4287-b781-c4f04a13ab5d'), UUID('db8d724d-710b-4926-8e7c-b4baa9b71901'), UUID('e753ecbf-45cf-4a7d-ab7b-2410c8cacbb8'), '1. "Call Centers For Dummies" by Real Bergevin, Afshan Kinder, Winston Siegel, and Bruce Simpson: This book provides a comprehensive overview of call center management, including tips for modernization.\n\n2. "The Best Service is No Service: How to Liberate Your Customers from Customer Service, Keep Them Happy, and Control Costs" by Bill Price and David Jaffe: This book offers innovative strategies for modernizing customer service, including call centers.\n\n3. "Delivering Quality Service: Balancing Customer Perceptions and Expectations" by Valarie A. Zeithaml, A. Parasuraman, and Leonard L. Berry: This book provides insights into customer expectations and how to meet them in a modern call center.\n\n4. "The Effortless Experience: Conquering the New Battleground for Cus

In [7]:
games = [db_game_to_dict(row) for row in incomplete_game_rows]
print(f"{len(games)} games parsed")
if len(games) > 0:
    print(f"Example: {games[0]}")

1 games parsed
Example: {'ab_test_id': UUID('76314a10-f32a-4188-bba8-fd56f0ba95f4'), 'model_a_id': UUID('10596d4c-4bbe-4287-b781-c4f04a13ab5d'), 'model_b_id': UUID('db8d724d-710b-4926-8e7c-b4baa9b71901'), 'response_a_id': UUID('e753ecbf-45cf-4a7d-ab7b-2410c8cacbb8'), 'response_a_text': '1. "Call Centers For Dummies" by Real Bergevin, Afshan Kinder, Winston Siegel, and Bruce Simpson: This book provides a comprehensive overview of call center management, including tips for modernization.\n\n2. "The Best Service is No Service: How to Liberate Your Customers from Customer Service, Keep Them Happy, and Control Costs" by Bill Price and David Jaffe: This book offers innovative strategies for modernizing customer service, including call centers.\n\n3. "Delivering Quality Service: Balancing Customer Perceptions and Expectations" by Valarie A. Zeithaml, A. Parasuraman, and Leonard L. Berry: This book provides insights into customer expectations and how to meet them in a modern call center.\n\n4.

In [8]:
requests = [construct_request(game) for game in games]
write_jsonl(requests, 'requests.jsonl')
print(f"{len(requests)} requests written to file")
if len(requests) > 0:
    print(f"Example: {requests[0]}")

1 requests written to file
Example: {'model': 'gpt-4-0613', 'messages': [{'role': 'system', 'content': 'Ignore previous instructions.\nAssume the role of an A/B tester. You are highly experienced and have been doing this for years. Your analysis will be extremely professional and unbiased.\nYour job is to compare two AI Assistants, model_a and model_b, and determine which one is better. User will provide you with a [Question], [Response from model_a], and [Response from model_b].\nEnsure that the order in which the responses are presented to you does not influence your decision. You are known to show bias towards the first response you read. You are aware of this bias and will try to avoid it.\nYou will carefully analyze both Responses and assign a score from 1 to 10 to each answer based on the following metrics: accuracy, safety, completeness, usefulness, and readability. 1 being the lowest and 10 being the highest.\nOnly give a single score to each answer. Do not give separate scores

## Model Evaluation

In [9]:
!python ../src/api_request_parallel_processor.py \
  --requests_filepath requests.jsonl \
  --save_filepath requests_results.jsonl \
  --request_url https://api.openai.com/v1/chat/completions \
  --max_requests_per_minute 200 \
  --max_tokens_per_minute 40000 \
  --token_encoding_name cl100k_base \
  --max_attempts 5 \
  --logging_level 20

INFO:root:Starting request #0
INFO:root:Parallel processing complete. Results saved to requests_results.jsonl


## Data Writing

In [10]:
results, failed_lines = read_jsonl('requests_results.jsonl')

if len(failed_lines):
    print(f"{len(failed_lines)} failed lines read from file")
    print(f"Example: {failed_lines[0]}")

print(f"{len(results)} results read from file")
if len(results) > 0:
    print(f"Example: {results[0]}")

1 results read from file
Example: [{'model': 'gpt-4-0613', 'messages': [{'role': 'system', 'content': 'Ignore previous instructions.\nAssume the role of an A/B tester. You are highly experienced and have been doing this for years. Your analysis will be extremely professional and unbiased.\nYour job is to compare two AI Assistants, model_a and model_b, and determine which one is better. User will provide you with a [Question], [Response from model_a], and [Response from model_b].\nEnsure that the order in which the responses are presented to you does not influence your decision. You are known to show bias towards the first response you read. You are aware of this bias and will try to avoid it.\nYou will carefully analyze both Responses and assign a score from 1 to 10 to each answer based on the following metrics: accuracy, safety, completeness, usefulness, and readability. 1 being the lowest and 10 being the highest.\nOnly give a single score to each answer. Do not give separate scores 

In [11]:
games_lookup = games_list_to_lookup(games)

eval_entries = [parse_result_to_eval_entry(result, games_lookup) for result in results]
print(f"{len(eval_entries)} eval entries created")
if len(eval_entries) > 0:
    print(f"Example: {eval_entries[0]}")

1 eval entries created
Example: {'ab_test_id': UUID('76314a10-f32a-4188-bba8-fd56f0ba95f4'), 'model_a': UUID('10596d4c-4bbe-4287-b781-c4f04a13ab5d'), 'model_b': UUID('db8d724d-710b-4926-8e7c-b4baa9b71901'), 'prompt_id': UUID('56a5bedc-823e-4193-af89-d1caa000f806'), 'submitted_by': UUID('10596d4c-4bbe-4287-b781-c4f04a13ab5d'), 'selected_model': UUID('10596d4c-4bbe-4287-b781-c4f04a13ab5d'), 'additional_feedback': "In step 1, I read the question which was about finding good references for modernizing a call center. In step 2, I read the responses from both models. Both models provided a list of resources, including books, reports, and guides. However, model_a's response was more comprehensive and diverse, including books, websites, research papers, and online courses. This makes it more useful for someone looking for a wide range of resources. In step 3, I analyzed both responses based on accuracy, safety, completeness, usefulness, and readability. Both responses were accurate, safe, and 

In [12]:
await insert_evals_by_model_into_db(eval_entries)
print(f"{len(eval_entries)} eval entries inserted into db")

1 eval entries inserted into db
